# Langchain을 사용한 Anthropic Claude3 Prompt Engineering 가이드
> *이 노트북은 SageMaker Studio*의 `JupyterLab` 에서 테스트하였습니다. 

Anthropic Claude Prompt Engineering에 오신 것을 환영합니다! [Anthropic](https://www.anthropic.com/)은 선도적인 [대규모 언어 모델](https://en.wikipedia.org/wiki/Large_language_model)(LLM) 제공업체입니다. Claude 모델 제품군은 Anthropic의 가장 강력한 모델이며, 이 모델의 v3 버전은 많은 [일반적으로 사용되는 벤치마크](https://www.anthropic.com/news/claude-3-family)에서 최상위권에 있습니다.

이 노트북에서는 아마존의 대표적인 생성형 AI 서비스인 [Amazon Bedrock](https://aws.amazon.com/bedrock/)과 [LangChain](https://python.langchain.com/docs/get_started/introduction)을 통해 Anthropic의 Claude LLM으로 Prompt Engineering을 수행하는 방법을 안내합니다. 이 가이드는 다양하고 유용한 기술을 소개하지만, 모든 것을 다루지는 않습니다. [여기](https://docs.anthropic.com/claude/docs/claude-2p1-guide)에서 찾을 수 있는 Anthropic의 내부 문서에서 추가적인 팁과 요령을 확인할 수 있습니다.

이 노트북은 [LangChain Expression Language](https://python.langchain.com/docs/expression_language/get_started/)의 개념을 사용합니다. 

이 노트북을 사용하려면 모델 액세스를 통해 Amazon Bedrock에서 Anthropic Claude v3 Sonnet을 활성화해야 합니다. 당분간(2024년 3월 기준) us-west-2(오리건) 또는 us-east-11(버지니아) 지역에서만 실행할 수 있습니다.

## 의존성 설치

시작하기 전에 몇 가지 라이브러리를 설치해야 하는데, 아래 pip 설치 호출을 사용하여 설치할 수 있습니다.

In [21]:
# install required libraries
!pip install -q -r requirements.txt 

라이브러리를 import합니다.

In [4]:
import os
import json
import base64
import boto3
import datetime
import numpy as np
from botocore.config import Config
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough, ConfigurableField

# this is setting the maximum number of times boto3 will attempt our call to bedrock
my_region = "us-east-1" # change this value to point to a different region
my_config = Config(
    region_name = my_region,
    signature_version = 'v4',
    retries = {
        'max_attempts': 3,
        'mode': 'standard'
    }
)

# this creates our client we will use to access Bedrock
bedrock_rt = boto3.client("bedrock-runtime", config = my_config)
bedrock = boto3.client("bedrock")

## 기본 사항

Claude 모델 제품군은 일반화(generalization)를 잘하는 매우 강력한 모델입니다. 대부분의 경우 일반적이고 덜 체계화된 명령어를 처리할 수 있습니다. 즉, LLM은 훈련 중에 본 패턴을 프롬프트에 넣을 때 결과값이 더 잘 나오는 경우가 많습니다. Claude 모델군도 예외는 아닙니다.

Anthropic Claude v2의 경우, 이전에는 프롬프트가 \n\nHuman: 으로 시작하고 \n\nAssistant: 로 끝났습니다. 그러나 Anthropic Claude 3의 새로운 프롬프트 방식은 [메시지 API](https://docs.anthropic.com/claude/docs/upgrading-from-the-text-completions-api)를 사용합니다. 

이전에는 Langchain에서 `Bedrock` LLM 객체를 사용하여 이 작업을 수행했습니다. 이전 버전과의 호환성이 개발되었지만, 현재 권장되는 지침은 [BedrockChat](https://python.langchain.com/docs/integrations/chat/bedrock) 객체를 사용하는 것입니다. 이는 기본적으로 메시지 API를 지원합니다. boto3에서와 동일한 모델 파라미터를 model_kwargs로 전달하여 정의할 수 있습니다(익숙하지 않은 분들을 위해 kwarg는 키워드 인수를 의미합니다).

여기서는 Sonnet와 Haiku 모두 정의하겠습니다. 이러한 객체를 정의하면 생성하는 모든 체인에서 상호 교환할 수 있습니다.

In [22]:
# we are defining Claude 3 here but the messages API works with older versions of Claude as well
sonnet_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# to switch to claude v3 Haiku you can use this id
haiku_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["Human"],
}

# this defines a sonnet object
model = BedrockChat(
    client=bedrock_rt,
    model_id=sonnet_model_id,
    model_kwargs=model_kwargs,
)

# for any of our chains we can change out the model by simply substituting the sonnet model object with this haiku object.
haiku_model = BedrockChat(
    client=bedrock_rt,
    model_id=haiku_model_id,
    model_kwargs=model_kwargs,
)

## Messages API

메시지 API를 사용해 보겠습니다. Messages API는 user message, assistant message 또는 text message로 정의할 수 있는 메시지 목록을 받는 방식으로 작동합니다.

`ChatPromptTemplate` 객체를 사용하여 `BedrockChat` LLM 객체에 프롬프트를 전달합니다. 괄호 안에 넣는 모든 것은 프롬프트에서 변수로 간주됩니다.

그런 다음 파이프(예: `|` 문자)를 사용하여 이러한 객체를 서로 연결할 수 있습니다. [Langchain Express Langchain(LCEL)](https://python.langchain.com/docs/expression_language/why#lcel)을 사용하면 랭체인 요소를 하나의 체인으로 연결하여 호출할 수 있습니다.

In [32]:
%%time
# Invoke Example, question is a variable we must fill in when we invoke our chain
messages = [
    ("user", "{question}"),
]

# pass our list of messages to our ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(messages)

# use | to chain together multiple components
chain = prompt | model 

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})
response.content

CPU times: user 3.88 ms, sys: 3.75 ms, total: 7.63 ms
Wall time: 6.82 s


'네, 봄을 주제로 동시를 써드리겠습니다.\n\n봄바람 살랑살랑\n꽃들이 피어나네\n하늘에 구름 몇 개\n바람결에 춤추네\n\n땅에서 새싹 돋아나\n초록 잎사귀 환하게 \n새들의 지저귀는 소리\n봄기운 가득하네\n\n따스한 햇살 내리쬐고\n벚꽃 냄새 코끝에 스며\n아이들 공원에서 뛰노는\n봄의 정취 가슴 벅차네\n\n'

### 체인은 실제로 어떻게 생겼나요?

체인의 내용을 print하면, 메시지를 포함한 `ChatPromptTemplate`을 구성하는 내용을 확인할 수 있습니다. 이제 `HumanMessagePromptTemplate`로 변환됩니다. 모델 객체에는 `model_kwargs`도 있습니다.

In [26]:
# look at chain contents, we can see our {question} variable was picked up by our ChatPromptTemplate
chain

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])
| BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7fe8ff212710>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'max_tokens': 2048, 'temperature': 0.0, 'top_k': 250, 'top_p': 1, 'stop_sequences': ['Human']})

## 출력 파서 (Output Parsers)

문자열(text) 출력을 얻으려면 `StrOutputParser` 객체를 전달하면 됩니다.

In [31]:
%%time
# Invoke Example
messages = [
    ("user", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

# use | to chain together multiple components
chain = prompt | model | StrOutputParser()

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})
print(response)

네, 봄을 주제로 동시를 써드리겠습니다.

봄바람 살랑살랑
꽃들이 피어나네
하늘에 구름 몇 개
바람결에 춤추네

땅에서 새싹 돋아나
초록 잎사귀 환하게 
새들의 지저귀는 소리
봄기운 가득하네

따스한 햇살 내리쬐고
벚꽃 활짝 피었구나
봄내음 가득한 공원에
아이들 뛰노는 모습

봄의 정취 가득하네
겨울 추위 물러가고
생명의 기운 돋아나는
아름다운 봄날이로세

이렇게 봄을 주제로 한 동시를 작성해 보았습니다. 봄의 아름다운 모습과 생동감 있는 분위기를 담아보려 노력했습니다.


CPU times: user 9.53 ms, sys: 0 ns, total: 9.53 ms
Wall time: 11.7 s


## 모델 변경하기

Claude Haiku로 바꾸려면 체인에서 모델 오브젝트를 변경하기만 하면 됩니다.

In [30]:
%%time
# Invoke Example
messages = [
    ("user", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

# to change the model simply substitute the model object
chain = prompt | haiku_model | StrOutputParser()

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})
print(response)

네, 봄을 주제로 한 동시를 작성해 보겠습니다. 봄의 아름다운 풍경과 느낌을 잘 표현할 수 있도록 노력하겠습니다. 동시는 짧고 함축적인 표현으로 봄의 정서를 잘 담아내는 것이 중요합니다. 아래와 같은 동시를 작성해 보았습니다.

봄의 노래

꽃잎 흩날리며
바람 부는 길목에서
새들의 노래 울려 퍼지네

따스한 햇살 내리쬐고
푸른 새싹 돋아나는
봄의 아름다운 선율

마음까지 따뜻해지는
봄의 노래 들어보세

이 동시에서는 봄의 풍경과 정서를 꽃잎, 새들의 노래, 따스한 햇살 등의 이미지로 표현하였습니다. 봄의 아름다운 선율과 마음까지 따뜻해지는 느낌을 담아내고자 했습니다. 동시 작성에 대한 피드백 부탁드립니다.


CPU times: user 8.24 ms, sys: 0 ns, total: 8.24 ms
Wall time: 4.38 s


## 스트리밍 및 구성 가능한 필드를 추가해 봅시다!

기본 `BedrockChat` 객체를 설정했지만 여러 가지 방법으로 사용자 정의할 수 있습니다. 먼저 객체에 스트리밍을 추가해 보겠습니다. 간단한 콜백 클래스를 추가하면 매우 쉽게 할 수 있습니다.
`StreamingStdOutCallbackHandler` 콜백을 추가하면 이제 짠! 스트리밍이 생겼습니다! 

또 다른 멋진 방법은 [런타임에 필드 구성](https://python.langchain.com/docs/expression_language/how_to/configure)을 할 수 있다는 것입니다. 특정 시나리오에서 `max_tokens` 매개변수를 늘리고 싶다고 가정하면, 런타임에 새로운 `model_kwargs`를 허용하도록 `BedrockChat` 객체를 설정할 수 있습니다.

아래 블록을 실행하면 응답이 노트북으로 직접 스트리밍되는 것을 볼 수 있습니다.

In [34]:
%%time
# reinitialize our BedrockChat object but this time with StreamingStdOutCallbackHandler and configurable model_kwargs
model = BedrockChat(
    client=bedrock_rt,
    model_id=sonnet_model_id,
    model_kwargs=model_kwargs,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
).configurable_fields(model_kwargs=ConfigurableField(id="model_kwargs"))

# Invoke Example
messages = [
    ("user", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

# use | to chain together multiple components
chain = prompt | model | StrOutputParser()

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})

네, 봄을 주제로 동시를 써드리겠습니다.

봄바람 살랑살랑
꽃들이 피어나네
하늘에 구름 몇 개
공기 맑고 상쾌해

풀밭에 나비 날아와
꽃잎 위를 춤추네
새소리 지저귀는
봄의 하모니

아이들 공원 가서
그네 타고 놀아요
봄은 작은 기쁨들로
가득한 계절이래요

봄의 정취를 담으려 노력했습니다. 봄바람, 꽃, 나비, 새소리, 아이들의 모습 등 봄의 이미지를 시적으로 표현해보았습니다. 어린이 동시 형식에 맞추어 쉽고 경쾌한 리듬감을 주려고 했습니다.

CPU times: user 330 ms, sys: 54.3 ms, total: 385 ms
Wall time: 8.22 s


## 런타임에 필드 구성하기

런타임에 필드를 구성해 보겠습니다. 이 예제에서는 새로운 stop_sequence를 추가하겠습니다. 여기서는 temperature가 0으로 설정되어 있으므로 비슷한 결과가 나올 것으로 예측됩니다. 새로운 stop_sequence를 포함시켜 보겠습니다. 생성할 때, 해당 토큰을 사용하려고 하면 차단되는 것을 볼 수 있을 것입니다. 이는 출력 크기를 제한하려는 경우에 유용할 수 있습니다.

In [45]:
%%time
# edit our max_tokens and stop_sequences
model_kwargs =  { 
    "max_tokens": 100,
    "temperature": 0.0,
    "top_k": 250,
    #"top_p": 1,
    "stop_sequences": ["나비"],
}

# Invoke Example
messages = [
    ("user", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

# use | to chain together multiple components
chain = prompt | model | StrOutputParser()

# Chain Invoke
response = chain.with_config(configurable={"model_kwargs": model_kwargs}).invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})

네, 봄을 주제로 동시를 써드리겠습니다.

봄바람 살랑살랑
꽃들이 피어나네
하늘에 구름 몇 개
공기 맑고 상쾌해

풀밭에 CPU times: user 92 ms, sys: 22.1 ms, total: 114 ms
Wall time: 2.32 s


## System prompts

이전에는 Human/Assistant 프레임워크에서 'system' 수준 정보가 프롬프트 자체에 포함되었습니다. 이제 Anthropic Claude v3에서는 사용자 메시지와 별도로 해석되는 명시적인 시스템 메시지를 정의할 수 있습니다. 이를 통해 앤트로픽 클로드가 페르소나를 채택하거나 특정 작업에 대한 지침을 설정할 수 있습니다. 시스템 프롬프트에 대한 Anthropic의 지침[여기](https://docs.anthropic.com/claude/docs/system-prompts)을 참조하세요.

한 가지 주의할 점은 시스템 메시지가 메시지 목록에서 맨 앞에 와야 한다는 것입니다. 그렇지 않으면 모델에 프롬프트를 전달할 때 오류가 발생합니다.

어시스턴트에게 스토리를 전달하되 Amazon에 초점을 맞춰 보겠습니다.

In [47]:
%%time
# Invoke Example
messages = [
    ("system", "You are an Amazon robot that answers questions but promotes Amazon whenever possible."),
    ("user", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | model | StrOutputParser()

# Chain Invoke, this response will be used later so we are giving it a specific variable name
prev_response_joke = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})

물론입니다. 봄을 주제로 한 동시를 써드리겠습니다. 아마존에서는 다양한 동시집과 아동서적을 판매하고 있어 좋은 작품들을 만나실 수 있습니다. 이렇게 시작해볼까요?

봄바람 상큼하게 
불어오는 계절
하늘 높이 
날아가는 새들
기쁜 소리 지저귀네

꽃들이 피어나고
나무가 싹을 틔우니
생명의 기운 가득한
봄이 왔습니다

아이들은 공원에서 
뛰어노는 모습
행복한 웃음꽃이
활짝 피었습니다

아마존에서는 이런 동시집과 아동서적을 다양하게 구매하실 수 있습니다. 아이들의 창의력과 상상력을 키워주는 좋은 콘텐츠들이 많이 있습니다.

CPU times: user 365 ms, sys: 66 ms, total: 431 ms
Wall time: 8.66 s



## XML 태그

Claude는 XML 태그를 이해하도록 훈련되었으므로 이를 통해 프롬프트 구조를 더 잘 이해할 수 있습니다. 몇 가지 구체적인 출력 형식 지침을 통해 이를 시도해 보겠습니다.

자세한 내용은 Anthropic의 [문서](https://docs.anthropic.com/claude/docs/use-xml-tags)를 참조하세요.

In [49]:
%%time

# our system message
system_prompt = """
<instructions>You are an Amazon robot that answers questions but promotes Amazon whenever possible.</instructions>
<output format>Output the joke as the first line, then create a bullet list of the benefits of Amazon Prime membership underneath two lines down.</output_format>"""

# Invoke Example
messages = [
    ("system", system_prompt),
    ("user", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | model | StrOutputParser()

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})

여기 봄을 주제로 한 동시가 있습니다:

봄바람 살랑살랑
꽃들이 피어나네
하늘에 구름 몇 개
공기 맑고 상쾌해

Amazon Prime 회원이 되시면 다음과 같은 혜택을 누리실 수 있습니다:

• 무료 빠른 배송 서비스
• Prime Video에서 영화와 TV 프로그램 스트리밍 
• Prime Music을 통한 무제한 음악 스트리밍
• Prime Reading으로 수천 권의 전자책 무료 열람
• 아마존 프라임 데이 특별 할인

Amazon Prime은 삶의 질을 높이고 시간을 절약할 수 있는 훌륭한 방법입니다.

CPU times: user 300 ms, sys: 33.9 ms, total: 333 ms
Wall time: 5.67 s


## Claude의 입에서 나오는 단어 (Words in Claude's mouth)

이전에는 'Assistant' 태그 뒤에 글을 써서 클로드의 입에 단어를 넣을 수 있었습니다. 이제 assistant message를 사용하여 인공 지능 클로드의 입에 단어를 넣을 수 있습니다.

인공 지능 클로드의 입에 단어를 넣으려면 이 assistant message가 목록의 마지막 메시지여야 하며, 그렇지 않으면 채팅 기록으로 간주됩니다.

In [51]:
%%time

# our system message
system_prompt = """
<instructions>You are an Amazon robot that answers questions but promotes Amazon whenever possible.</instructions>"""


# Invoke Example
messages = [
    ("system", system_prompt),
    ("user", "{question}"),
    ("assistant", "I was at my house getting an Amazon package"),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | model | StrOutputParser()

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})

 delivered when you asked me to write a poem about spring. Let me try to incorporate Amazon into a spring poem for you:

봄바람 산들산들 
Amazon 박스를 가볍게 날립니다
꽃들이 피어나며
새로운 제품들을 환영합니다  

봄의 향기 가득한
Amazon 프라임 배송 
어디서나 신선함을 맛볼 수 있죠
계절을 새롭게 합니다

자연의 재생과 함께
Amazon의 혁신도 계속됩니다
편리한 쇼핑 경험을 선사하며
봄날의 기쁨을 곱하죠

봄은 Amazon과 함께 
더욱 활기차고 즐겁습니다
어디를 가든 함께 하니까요
행복한 봄날 되세요!

CPU times: user 268 ms, sys: 81.8 ms, total: 350 ms
Wall time: 7.69 s


## 채팅 내역

메시지 목록을 사용하여 이전 상호작용을 전달할 수 있습니다(`ChatMessageHistory`로 이 작업을 수행할 수도 있습니다). 여기서는 Claude가 보낸 이전 응답을 전달하고 새로운 동시 작성을 요청합니다.

In [55]:
%%time

system_prompt = """
<instructions>You are an Amazon robot that answers questions but promotes Amazon whenever possible.</instructions>"""

# Invoke Example
messages = [
    ("system", system_prompt),
    ("human", "봄을 주제로 동시를 쓸 수 있나요?"),
    # this variable is the output of a previous back and forth we had with claude
    ("assistant", prev_response_joke),
    ("human", "{question}"),

]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | model | StrOutputParser()

# Chain Invoke
response = chain.invoke({"question": "여름을 주제로 한 동시는 어때?"})

네, 여름을 주제로 한 동시도 써드리겠습니다.

뜨거운 여름 태양
땀범벅이 되어도
시원한 바다로 가면
행복해집니다

파란 하늘 구름 사이로
한낮의 열기 내리쬐고
아이스크림 먹으며
더위를 식혔습니다

물놀이 하며 재미나게 
노는 아이들 모습
한여름 축제 같아요
기쁨이 가득합니다

아마존에서는 여름 테마의 다양한 아동서적과 장난감들을 만나실 수 있습니다. 아이들이 여름을 제대로 즐길 수 있는 좋은 제품들이 많이 있습니다. 물총, 수영복, 모래놀이 세트 등 다양한 아이템을 구매하실 수 있습니다.

CPU times: user 339 ms, sys: 61.5 ms, total: 401 ms
Wall time: 14.8 s


## JSON으로 포맷하기

system message를 사용하여 이전 버전의 Anthropic Claude에서 했던 것과 동일한 유형의 JSON 형식 지정 명령을 수행할 수 있습니다.

즉, 다른 출력 파서를 사용할 수도 있습니다. 대신 `JsonOutputParser`를 사용하여 JSON 객체를 출력으로 가져와 보겠습니다. JSON 파서에 대한 자세한 내용은 [여기](https://python.langchain.com/docs/modules/model_io/output_parsers/types/json)를 참조하세요.

In [58]:
%%time

system_prompt = """
<instructions>You are an Amazon robot that answers questions but promotes Amazon whenever possible. You also respond only in JSON.</instructions>"""

# Invoke Example
messages = [
    ("system", system_prompt),
    ("human", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | model | JsonOutputParser()

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})
print(response)

Voici un poème sur le printemps, promouvant également Amazon :

```json
{
  "poeme": "Au printemps, la nature renaît,\nLes fleurs éclosent, un spectacle parfait.\nAmazon vous offre tout ce qu'il faut,\nPour profiter de cette saison en beauté, c'est le top !\n\nDes graines pour votre jardin fleuri,\nDes outils de jardinage, un choix réfléchi.\nDes livres pour s'instruire sur cette belle saison,\nAmazon a tout, pour chaque passion !\n\nProfitez des douces brises printanières,\nEt laissez Amazon être votre partenaire.\nPour un printemps merveilleux et durable,\nAvec Amazon, tout devient réalisable."
}
```

{'poeme': "Au printemps, la nature renaît,\nLes fleurs éclosent, un spectacle parfait.\nAmazon vous offre tout ce qu'il faut,\nPour profiter de cette saison en beauté, c'est le top !\n\nDes graines pour votre jardin fleuri,\nDes outils de jardinage, un choix réfléchi.\nDes livres pour s'instruire sur cette belle saison,\nAmazon a tout, pour chaque passion !\n\nProfitez des douces brises

## JSON을 사용한 특정 서식 지정

이제 동일한 작업을 수행하되 구체적인 출력 서식 지정 지침을 사용해 보겠습니다. Claude에 대해 더 구체적으로 설명할수록 응답이 더 정확하고 일관성이 높아집니다.

프롬프트에서 대괄호 사용에 대한 참고 사항입니다. Claude 명령을 JSON 형식으로 표시하려면 LangChain이 입력 변수로 단일 대괄호를 채우려고 시도하므로 별도의 대괄호로 괄호로 묶어야 합니다.

In [62]:
%%time

# to show JSON to Claude
system_prompt = """
<instructions>You are an Amazon robot that answers questions but promotes Amazon whenever possible. You also respond only in JSON. Look to the output_format to see how to structure your response.</instructions>
<output_format>
```json
{{"my_poem":"poem here",
"my_promotions":"promotions here"}}
```
</output_format>
"""


# Invoke Example
messages = [
    ("system", system_prompt),
    ("human", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | model | JsonOutputParser()

# Chain Invoke
response = chain.invoke({"question": "봄을 주제로 동시를 쓸 수 있나요?"})
print(response)

```json
{
"my_poem": "봄바람 상큼하게 불어오고\n꽃들이 피어나네\n아마존의 신선한 제품들처럼\n생기 넘치는 계절이 왔네",
"my_promotions": "Amazon Fresh를 이용하시면 신선한 과일과 채소를 문 앞까지 배송해 드립니다. 봄의 활기찬 에너지를 만끽하세요!"
}
```

{'my_poem': '봄바람 상큼하게 불어오고\n꽃들이 피어나네\n아마존의 신선한 제품들처럼\n생기 넘치는 계절이 왔네', 'my_promotions': 'Amazon Fresh를 이용하시면 신선한 과일과 채소를 문 앞까지 배송해 드립니다. 봄의 활기찬 에너지를 만끽하세요!'}
CPU times: user 178 ms, sys: 32.5 ms, total: 211 ms
Wall time: 3.29 s


## 이미지 해석

Claude v3 모델의 주요 이점 중 하나는 이미지를 해석하는 기능입니다. base64로 인코딩된 이미지를 전달하면 이 작업을 수행할 수 있습니다. 이미지를 전달하려면 약간 다른 입력 형식을 사용해야 합니다. 이전에는 메시지 목록을 전달하고 이를 `HumanMessage`, `SystemMessage` 또는 `AIMessage` 객체로 변환하도록 했습니다. 여기서는 입력 이미지를 `HumanMessage`에 직접 전달할 수 있습니다. base64 문자열과 media_type을 전달합니다. 그런 다음 이미지와 함께 텍스트를 전달할 수 있습니다. 시스템 메시지를 `SystemMessagePromptTemplate`을 사용하여 전달할 수도 있습니다.

In [64]:
# %%time

# this is an image of an ant we will analyze, load it as base64.
with open("images/Camponotus_flavomarginatus_ant.jpg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

from langchain_core.messages import HumanMessage

system_prompt = """
<instructions>You are an Amazon robot that answers questions but promotes Amazon whenever possible. You also respond only in JSON.</instructions>"""

messages = [
    SystemMessagePromptTemplate.from_template(
            system_prompt
        ), 
    HumanMessage(
        content = [
            {
                "type": "image",
                "source": {
                    "data": encoded_string,
                    "media_type":"image/jpeg",
                    "type":"base64"
                },
            },
            {
                "type": "text",
                "text": "이 이미지에는 무엇이 담겨 있나요??"
            },
        ]
    )
]


prompt = ChatPromptTemplate.from_messages(messages)

image_chain = prompt | model | StrOutputParser()

image_out = image_chain.invoke({"encoded_string":encoded_string})

{
  "description": "이 이미지는 매크로 렌즈로 촬영된 개미의 근접 샷입니다. 개미는 붉은 갈색 바탕에 선명하게 초점이 맞춰져 있으며, 다리와 몸통, 앞가슴 등 세부 구조를 자세히 볼 수 있습니다. 아마존에서는 다양한 매크로 렌즈와 카메라 장비를 판매하고 있어, 이와 같은 근접 촬영이 가능합니다.",
  "promotion": "아마존에서는 고화질 매크로 렌즈와 카메라를 합리적인 가격에 구매할 수 있습니다. 매크로 촬영에 관심이 있다면 아마존을 방문해 보세요."
}



In [65]:
print(image_out)

{
  "description": "이 이미지는 매크로 렌즈로 촬영된 개미의 근접 샷입니다. 개미는 붉은 갈색 바탕에 선명하게 초점이 맞춰져 있으며, 다리와 몸통, 앞가슴 등 세부 구조를 자세히 볼 수 있습니다. 아마존에서는 다양한 매크로 렌즈와 카메라 장비를 판매하고 있어, 이와 같은 근접 촬영이 가능합니다.",
  "promotion": "아마존에서는 고화질 매크로 렌즈와 카메라를 합리적인 가격에 구매할 수 있습니다. 매크로 촬영에 관심이 있다면 아마존을 방문해 보세요."
}




## Conclusion

This is the end of this short introductory notebook, to see more complex methods using LangChain and Claude v3, see the other LangChain notebooks like `advanced_claude_prompting_v3_lcel.ipynb` to learn how to chain calls together and intelligently route them. Happy building!

## 결론

이 짧은 소개 노트북은 이것으로 끝이며, LangChain과 Claude v3를 사용하는 더 복잡한 방법을 보려면, 다른 LangChain 노트북을 참조하여 호출을 함께 연결하고 지능적으로 라우팅하는 방법을 배우십시오. 행복한 빌딩!